In [ ]:

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> BASE DE DISCAGENS <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

import pandas as pd
import pyodbc
from datetime import datetime
import re

SERVER = r"TRC-DC-BDM\BD"
DATABASE = "SRC"

# Configuração da conexão com o banco de dados - CORRIGIDO
conn_str = (
    f'DRIVER={{SQL Server}};'
    f'SERVER={SERVER};'
    f'DATABASE={DATABASE};'
    f'Trusted_Connection=yes;'
)

# Caminho do arquivo SQL
sql_file_path = r'\\trc-dc-ad\Planejamento\MIS\CARTEIRAS\Ouze\Consultas\base_ouze.sql'

# Definir o mês desejado (setembro/2025)
dt_ini = '2025-09-01'
dt_fim = '2025-09-30'

def carregar_e_executar_sql(caminho_arquivo, data_inicio, data_fim):
    """
    Lê o arquivo SQL, substitui as datas e executa a consulta
    """
    try:
        # Ler o arquivo SQL
        with open(caminho_arquivo, 'r', encoding='utf-8') as file:
            sql_content = file.read()
        
        print(f"Arquivo SQL carregado: {caminho_arquivo}")
        
        # Substituir as datas no conteúdo do SQL
        sql_content = re.sub(
            r"DECLARE @DT_INI AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT_INI AS DATE = '{data_inicio}'",
            sql_content
        )
        
        sql_content = re.sub(
            r"DECLARE @DT_FIM AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT_FIM AS DATE = '{data_fim}'",
            sql_content
        )
        
        sql_content = re.sub(
            r"DECLARE @DT\s+AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT AS DATE = '{data_inicio}'",
            sql_content
        )
        
        sql_content = re.sub(
            r"DECLARE @DT2\s+AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT2 AS DATE = '{data_fim}'",
            sql_content
        )
        
        print(f"Datas ajustadas: {data_inicio} até {data_fim}")
        print("Conectando ao banco de dados...")
        
        # Conectar e executar
        conn = pyodbc.connect(conn_str)
        print("Conexão estabelecida. Executando consulta...")
        
        # Executar a query e carregar em DataFrame
        df_discagens = pd.read_sql(sql_content, conn)
        
        # Fechar conexão
        conn.close()
        
        print(f"\n✓ Dados carregados com sucesso!")
        print(f"Total de registros: {len(df_discagens):,}")
        print(f"\nColunas disponíveis: {list(df_discagens.columns)}")
        print(f"\nPrimeiras linhas:")
        print(df_discagens.head())
        
        return df_discagens
        
    except FileNotFoundError:
        print(f"❌ Erro: Arquivo não encontrado em {caminho_arquivo}")
        return None
    except pyodbc.Error as db_error:
        print(f"❌ Erro de banco de dados: {db_error}")
        return None
    except Exception as e:
        print(f"❌ Erro ao executar consulta: {e}")
        import traceback
        traceback.print_exc()
        return None

# Executar a função
print("Iniciando execução...")
df_discagens = carregar_e_executar_sql(sql_file_path, dt_ini, dt_fim)

# Verificar informações do DataFrame
if df_discagens is not None:
    print("\n" + "="*60)
    print("INFORMAÇÕES DO DATAFRAME")
    print("="*60)
    print(df_discagens.info())
    print(f"\nShape: {df_discagens.shape}")
else:
    print("\n❌ Falha ao carregar os dados.")

Iniciando execução...
Arquivo SQL carregado: \\trc-dc-ad\Planejamento\MIS\CARTEIRAS\Ouze\Consultas\base_ouze.sql
Datas ajustadas: 2025-09-01 até 2025-09-30
Conectando ao banco de dados...
Conexão estabelecida. Executando consulta...


C:\Users\claudiano.alves\AppData\Local\Temp\ipykernel_25724\3212311926.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_discagens = pd.read_sql(sql_content, conn)



✓ Dados carregados com sucesso!
Total de registros: 2,800,042

Colunas disponíveis: ['DATA', 'id', 'CONTRATO', 'CPF', 'ddd', 'fone', 'GrupoPrincipal', 'UltCodSigRecPublica', 'ResultadoClassificacao', 'MotivoEncerramentoBilhete', 'Instante200OKPub', 'Agente', 'tempoconversacao_ms']

Primeiras linhas:
         DATA   id    CONTRATO             CPF  ddd       fone  \
0  2025-09-01   66  48941489id  43640864000180   27  998267780   
1  2025-09-01  149  49012660id  57722707000129   11  945760003   
2  2025-09-01  196  47525229id  55280229000119   12  996448349   
3  2025-09-01  203  42643945id  54542547000148   45  999946130   
4  2025-09-01  220  41849079id  53330359000193   62  999623988   

   GrupoPrincipal UltCodSigRecPublica ResultadoClassificacao  \
0            4047                 480                          
1            4636                 480                          
2            4636                 404                          
3            4636                 404        

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>  classificação das discagens  <<<<<<<<<<<<<<<<<<<<<<<<

import numpy as np

conditions = [
    df_discagens['GrupoPrincipal'] == 4118,
    df_discagens['GrupoPrincipal'] == 4022,
    df_discagens['GrupoPrincipal'] == 4017,
    df_discagens['GrupoPrincipal'].isin([4047,4679,4681,4683,4671]),
    df_discagens['GrupoPrincipal'].isin([4433, 4504]),
    df_discagens['GrupoPrincipal'].isin([4326, 4636, 4637, 4649])
]

ddd_estado = {
    '11': 'SP', '12': 'SP', '13': 'SP', '14': 'SP', '15': 'SP', '16': 'SP', '17': 'SP', '18': 'SP', '19': 'SP',
    '21': 'RJ', '22': 'RJ', '24': 'RJ',
    '27': 'ES', '28': 'ES',
    '31': 'MG', '32': 'MG', '33': 'MG', '34': 'MG', '35': 'MG', '37': 'MG', '38': 'MG',
    '41': 'PR', '42': 'PR', '43': 'PR', '44': 'PR', '45': 'PR', '46': 'PR',
    '47': 'SC', '48': 'SC', '49': 'SC',
    '51': 'RS', '53': 'RS', '54': 'RS', '55': 'RS',
    '61': 'DF', '62': 'GO', '63': 'TO', '64': 'GO', '65': 'MT', '66': 'MT', '67': 'MS',
    '68': 'AC', '69': 'RO',
    '71': 'BA', '73': 'BA', '74': 'BA', '75': 'BA', '77': 'BA',
    '79': 'SE',
    '81': 'PE', '82': 'AL', '83': 'PB', '84': 'RN', '85': 'CE', '86': 'PI', '87': 'PE', '88': 'CE', '89': 'PI',
    '91': 'PA', '92': 'AM', '93': 'PA', '94': 'PA', '95': 'RR', '96': 'AP', '97': 'AM', '98': 'MA', '99': 'MA'
}


choices = [
    'ATIVO',
    'MANUAL',
    'RECEPTIVO',
    'URA CPC',
    'PREVENTIVO',
    'AGV NEGOCIADORA'
]

df_discagens['OPERACAO'] = np.select(conditions, choices, default='Outros')
df_discagens['ESTADO'] = df_discagens['ddd'].astype(str).str.zfill(2).map(ddd_estado)

In [65]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>> TESTES <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# tipo da coluna

# >>>>>>>>>>>>>>>>>>>>>>> VERIFICAR TIPO DE DADO DE UMA COLUNA ESPECÍFICA
#print(df_discagens['ddd'].dtype)
# primeiros valores (mostra índices também)
#df_discagens['ddd'].head(30)

# >>>>>>>>>>>>>>>>>>>>>>> MOSTRAR PARTE DO DATAFRAME
#df_discagens.head()

# >>>>>>>>>>>>>>>>>>>>>>> FILTRAR DADOS DE UMA COLUNA ESPECÍFICA
df_ativo = df_discagens[df_discagens['ESTADO'] == 'PR']
df_ativo.head()

# >>>>>>>>>>>>>>>>>>>>>>> CONTAR VALORES ÚNICOS DE UMA COLUNA ESPECÍFICA
# len(df_ativo)
# df_discagens['OPERACAO'].value_counts().reset_index().rename(columns={'index': 'OPERACAO', 'OPERACAO': 'QTD'})
#  print(filtro)
#len(df_discagens)
#print("Linhas:", df_discagens.shape[0], " | Colunas:", df_discagens.shape[1])
#df_discagens.columns.tolist()
#df_discagens['Agente'].unique()
#type(df_discagens)

# >>>>>>>>>>>>>>>>>>>>>>> resumo por estado
# df_resumo = (
#     df_discagens
#     .groupby('ESTADO')['CPF']
#     .nunique()
#     .reset_index()
#     .rename(columns={'CPF': 'QTD_CPF_DISTINTOS'})
#     .sort_values(by='QTD_CPF_DISTINTOS', ascending=False)
# )
# df_resumo.head()
# print(df_resumo)


,DATA,id,CONTRATO,CPF,ddd,fone,GrupoPrincipal,UltCodSigRecPublica,ResultadoClassificacao,MotivoEncerramentoBilhete,Instante200OKPub,Agente,tempoconversacao_ms,OPERACAO,ESTADO
3,2025-09-01,203,42643945id,54542547000148,45,999946130,4636,404,,foneinexistente,1899-12-30 00:00:00.0000000,,0,AGV NEGOCIADORA,PR
11,2025-09-01,330,50495714id,57588687000145,41,995070347,4636,486,,ocupado,1899-12-30 00:00:00.0000000,,0,AGV NEGOCIADORA,PR
74,2025-09-01,1017,48685417id,58454694000117,43,999957988,4636,480,,foneforadeservico,1899-12-30 00:00:00.0000000,,0,AGV NEGOCIADORA,PR
83,2025-09-01,1087,49013395id,58253861000161,41,985035076,4636,404,,foneinexistente,1899-12-30 00:00:00.0000000,,0,AGV NEGOCIADORA,PR
93,2025-09-01,1173,47329146id,22042009000106,42,999164407,4636,408,,naoatende,1899-12-30 00:00:00.0000000,,0,AGV NEGOCIADORA,PR
